In [2]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from skimage import exposure, measure
from scans import *
from tumor_centers import *
import os

In [3]:
def plot_slice(slice, title):
    plt.imshow(slice, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()
    
def load_data(scan_path, label_path):
    scan = nib.load(scan_path).get_fdata()
    label = nib.load(label_path).get_fdata()
    return scan, label

def select_slice(scan, label, slice_idx):
    return scan[:, :, slice_idx], label[:, :, slice_idx]

def preprocess_slice(slice):
    p2, p98 = np.percentile(slice, (2, 98))
    return exposure.rescale_intensity(slice, in_range=(p2, p98))

def region_growing(image, seed):
    seg_result = np.zeros_like(image)
    seg_result[seed] = 1
    points = [seed]
    seedval = image[seed]
    threshval = 0.2 * np.max(image)
    
    while points:
        x, y = points.pop(0)
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < image.shape[0] and 0 <= ny < image.shape[1]:
                    pointval = image[nx, ny]
                    if seg_result[nx, ny] == 0 and seedval - threshval <= pointval <= seedval + threshval:
                        seg_result[nx, ny] = 1
                        points.append((nx, ny))
    return seg_result

def dice_coefficient(segmented, ground_truth):
    intersection = np.logical_and(segmented, ground_truth)
    return (2. * intersection.sum()) / (segmented.sum() + ground_truth.sum())

In [4]:
dataset_path = 'D:\Dosyalar\dataset\Task06_Lung'
images_train_dir = os.path.join(dataset_path, 'imagesTr')
labels_train_dir = os.path.join(dataset_path, 'labelsTr')

In [5]:
with open('results.txt', 'w') as results_file:
    for scan_number in INDEXES[:21]:
        scan_path = os.path.join(images_train_dir, f'lung_{scan_number}.nii.gz')
        label_path = os.path.join(labels_train_dir, f'lung_{scan_number}.nii.gz')
        scan, label = load_data(scan_path, label_path)
        
        slice_indexes = globals()[f'scan_{scan_number}']
        tumor_centers = globals()[f'tumor_centers_{scan_number}']

        dscs = []
        sensitivities = []
        specificities = []
        accuracies = []
        weights = []

        for i, slice_idx in enumerate(slice_indexes):
            slice, slice_label = select_slice(scan, label, slice_idx)

            slice_preprocessed = preprocess_slice(slice)

            center = tumor_centers[i]
            seed_point = (int(center[1]), int(center[0])) # Swap x, y
            segmented_mask = region_growing(slice_preprocessed, seed_point)

            TP = np.sum(np.logical_and(segmented_mask == 1, slice_label == 1))
            TN = np.sum(np.logical_and(segmented_mask == 0, slice_label == 0))
            FP = np.sum(np.logical_and(segmented_mask == 1, slice_label == 0))
            FN = np.sum(np.logical_and(segmented_mask == 0, slice_label == 1))

            dsc = dice_coefficient(segmented_mask, slice_label)
            print(f'Scan {scan_number} - Slice {slice_idx}: {dsc}')
            sensitivity = TP / (TP + FN) if TP + FN > 0 else 0
            specificity = TN / (TN + FP) if TN + FP > 0 else 0
            accuracy = (TP + TN) / (TP + TN + FP + FN) if TP + TN + FP + FN > 0 else 0

            dscs.append(dsc)
            sensitivities.append(sensitivity)
            specificities.append(specificity)
            accuracies.append(accuracy)
            weights.append(np.sum(segmented_mask))

        weighted_mean_dsc = np.average(dscs, weights=weights)
        weighted_std_dsc = np.sqrt(np.average((dscs - weighted_mean_dsc) ** 2, weights=weights))
        mean_dsc = np.mean(dscs)  # Calculate mean DSC
        weighted_mean_sensitivity = np.average(sensitivities, weights=weights)
        weighted_std_sensitivity = np.sqrt(np.average((sensitivities - weighted_mean_sensitivity) ** 2, weights=weights))
        mean_sensitivity = np.mean(sensitivities)
        weighted_mean_specificity = np.average(specificities, weights=weights)
        weighted_std_specificity = np.sqrt(np.average((specificities - weighted_mean_specificity) ** 2, weights=weights))
        mean_specificity = np.mean(specificities)
        weighted_mean_accuracy = np.average(accuracies, weights=weights)
        weighted_std_accuracy = np.sqrt(np.average((accuracies - weighted_mean_accuracy) ** 2, weights=weights))
        mean_accuracy = np.mean(accuracies)
        
        results_file.write(f'Scan {scan_number}:\n'
                           f'  - Weighted Mean DSC: {weighted_mean_dsc}, '
                           f'Standard Deviation: {weighted_std_dsc}, Mean DSC: {mean_dsc}\n'
                           f'  - Weighted Mean Sensitivity: {weighted_mean_sensitivity}, '
                           f'Standard Deviation: {weighted_std_sensitivity}, Mean Sensitivity: {mean_sensitivity}\n'
                           f'  - Weighted Mean Specificity: {weighted_mean_specificity}, '
                           f'Standard Deviation: {weighted_std_specificity}, Mean Specificity: {mean_specificity}\n'
                           f'  - Weighted Mean Accuracy: {weighted_mean_accuracy}, '
                           f'Standard Deviation: {weighted_std_accuracy}, Mean Accuracy: {mean_accuracy}\n\n')

print("Results saved in 'results.txt'")

Scan 001 - Slice 231: 0.15384615384615385
Scan 001 - Slice 232: 0.34210526315789475
Scan 001 - Slice 233: 0.5535714285714286
Scan 001 - Slice 234: 0.7191780821917808
Scan 001 - Slice 235: 0.8411764705882353
Scan 001 - Slice 236: 0.8549222797927462
Scan 001 - Slice 237: 0.9072681704260651
Scan 001 - Slice 238: 0.8256880733944955
Scan 001 - Slice 239: 0.8277511961722488
Scan 001 - Slice 240: 0.8979591836734694
Scan 001 - Slice 241: 0.00324168712880868
Scan 001 - Slice 242: 0.002358251343949691
Scan 001 - Slice 243: 0.7885714285714286
Scan 001 - Slice 244: 0.6595744680851063
Scan 001 - Slice 245: 0.001940663547426627
Scan 001 - Slice 246: 0.6133333333333333
Scan 001 - Slice 247: 0.6625766871165644
Scan 001 - Slice 248: 0.22988505747126436
Scan 001 - Slice 250: 0.06666666666666667
Scan 003 - Slice 138: 0.5977011494252874
Scan 003 - Slice 139: 0.6113989637305699
Scan 003 - Slice 140: 0.8848920863309353
Scan 003 - Slice 141: 0.8922155688622755
Scan 003 - Slice 142: 0.8488063660477454
Scan 00